In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as osp
import matplotlib.pyplot as plt
import shutil
import runpy
import scipy

from sqlalchemy import create_engine
import mysql.connector

# MySQL connection

In [ ]:
try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="pathofpain"
    )
    print("Connection established")
    cursor = mydb.cursor(buffered=True)
    cursor.execute("use `SERS_ML_TEST`;")


except mysql.connector.Error as err:
    print("An error occurred:", err)

# MySQL DATABASE "SERS_ML_TEST" create

In [ ]:
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)

# Open the .sql file
sql_file = open('SERS-ML_test_createDB.sql','r')

# Read the .sql queries
sql_command = sql_file.read()

# Execute Multi statements in .sql script
for result in cursor.execute(sql_command, multi=True):
    print(result)

# Commit Change to database
mydb.commit()

# Directory Setting

In [ ]:
os. getcwd() 

In [ ]:
#Please enter the directory
data_dir = 'Exp_Data/'

In [ ]:
for dirpath, dirnames, filenames in os.walk('Exp_Data'):
    print(dirpath)
    print(dirnames)
    print(filenames)

In [ ]:
for dirpath, dirnames, filenames in os.walk('Exp_Data/'):
    print(dirpath)
    print(dirnames)
    print(filenames)

In [ ]:
de_ba_path = 'SERS_debaseline.py'

k = 0
for dirpath, dirnames, filenames in os.walk(data_dir):
    for filenames_index in filenames :
        print(filenames_index)
        folder_name = filenames_index.split('.')[0]
        label = folder_name
        # folder_name = folder_name + '_' + str(k)  
        print(folder_name)
        print(filenames_index)
        dir = dirpath + '/' + folder_name +'/'
        print(dir)
        os.makedirs(dir, exist_ok = True)

        file = dirpath + '/' + filenames_index
        dest = dir + '/' + label + '.txt'
        de_path = dir+'/' + 'debase.py'
        shutil.move(file, dest)
        shutil.copy(de_ba_path, de_path)


In [ ]:
k = 0
for dirpath, dirnames, filenames in os.walk(data_dir):
    for filenames_index in filenames :
        if filenames_index == 'debase.py':
            fi = dirpath + '/' + filenames_index
            file_globals = runpy.run_path(fi)


## RAW data record into database


In [ ]:
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")


k = 0
for dirpath, dirnames, filenames in os.walk(data_dir):
    if dirpath[-2:] == 'sb':
        
        df = pd.read_csv(dirpath + '/'+ filenames[0])
        count = df.shape[0]
        print(count)

        print(filenames[0])
        label = '_'.join(filenames[0].split("_")[1:-3])
        print(label)

        info = label.split("_")
        print(info)

        bacteria = info[0].split("-")[0]
        print (bacteria)

        strain = info[0].split("-")[1]
        print(strain)

        subtype = 'WT'
        if (len(info[0].split("-")) == 3):
            subtype = info[0].split("-")[2]
        print(subtype)


        antibiotic = info[1][0:3] 
        print(antibiotic)

        anti_conc = info[1][3:] 
        print(anti_conc)
        
        anti_time =  info[2]
        print(anti_time)

        sql_command = """INSERT INTO `Lab_Groups`(`file_name`,`bacteria`,`strain`,`subtype`,`antibiotic`, `anti_conc`,`anti_time`, `data_path`,`count`)
               VALUES ('{file_name}','{bacteria}','{strain}','{subtype}','{antibiotic}', '{anti_conc}','{anti_time}', '{data_path}','{count}');""".format(
                file_name = filenames[0], bacteria = bacteria, strain=strain, subtype=subtype, antibiotic=antibiotic, anti_conc=anti_conc,
                            anti_time = anti_time, data_path = dirpath + '/' + filenames[0], count = count
               )


        print (sql_command)
        cursor.execute(sql_command)
        mydb.commit()
        


In [ ]:

cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")


antibiotic = 'Amp'
anti_conc = 16
anti_time = '30min'


command = ("""SELECT `data_path`, `file_name`, `id` FROM `Lab_Groups`
            WHERE (`antibiotic` = '{antibiotic}') AND (`anti_conc` = '{anti_conc}') AND (`anti_time` = '{anti_time}');""".format(
            antibiotic = antibiotic, anti_conc = anti_conc, anti_time = anti_time
))

cursor.execute(command)
groups_list = cursor.fetchall()
print(groups_list)

wave = np.arange(400, 1550.1, 0.5, dtype=float)
feature = np.zeros((1,2301))
labels = []
lab_groups = []




for group_file, filename, group_id in groups_list:
    print(group_file)
    print(filename)
    print(group_id)
        
    df = pd.read_csv(group_file)
    df = df.loc[:, '400':'1550']
    df = df.to_numpy()
    feature = np.row_stack((feature, df))
    label = '_'.join(filename.split("_")[1:-3])
    print(label)
    labels.extend([label]*df.shape[0])




feature = feature[1:,:]
print(feature.shape)
df = pd.DataFrame(feature, columns=wave)
df.insert(0, 'label', labels)
count = df.shape[0]
df.to_csv(data_dir + 'RAW_SERS_dataset.csv', index= False)


## Record RAW dataset into Database
sql_command = """INSERT INTO `Combine_SERS`(`file_name`,`antibiotic`, `anti_conc`,`anti_time`, `data_path`,`count`)
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', '{data_path}','{count}');""".format(
                file_name = 'RAW_SERS_dataset.csv', antibiotic=antibiotic, anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'RAW_SERS_dataset.csv', count = count)

print (sql_command)
cursor.execute(sql_command)
mydb.commit()



## Record relationship of lab_groups and combine_dataset in database
cursor.execute("""SELECT `id` FROM `Combine_SERS`
        WHERE (`antibiotic` = '{antibiotic}') AND (`anti_conc` = '{anti_conc}') AND (`anti_time` = '{anti_time}');""".format(
        antibiotic = antibiotic, anti_conc = anti_conc, anti_time = anti_time))
dataset_id = cursor.fetchall()[0][0]

for group_file, filename, group_id in groups_list:
    print(group_id)
    cursor.execute("""INSERT INTO `Combine_Groups`(`combine_id`, `lab_group_id`)
                   VALUES('{dataset_id}', '{group_id}');""".format(
                    dataset_id = dataset_id, group_id = group_id))
    mydb.commit()





## Data Cleaning by 733/690 ratio >= 1.2

In [ ]:
# Data Cleaning by 733/690 ratio >= 1.2

# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")

# experimental conditions
antibiotic = 'Amp'
anti_conc = 16
anti_time = '30min'


cursor.execute("""SELECT `data_path`,`id` FROM `Combine_SERS`
        WHERE (`antibiotic` = '{antibiotic}') AND (`anti_conc` = '{anti_conc}') AND (`anti_time` = '{anti_time}');""".format(
        antibiotic = antibiotic, anti_conc = anti_conc, anti_time = anti_time))
RAW_dataset = cursor.fetchall()
RAW_file = RAW_dataset[0][0]
RAW_id  = RAW_dataset[0][1]


# df = pd.read_csv(data_dir + 'RAW_SERS_dataset.csv', header = 0)
df = pd.read_csv(RAW_file, header = 0)

## Cleaning 
substrate_area = df.loc[:, '680.0':'700.0']
characteristic_area = df.loc[:, '720.0':'760.0']
substrate_peak = substrate_area.max(axis=1).to_numpy()
characteristic_peak = characteristic_area.max(axis=1).to_numpy()
peak_ratio = characteristic_peak / substrate_peak
peak_ratio = peak_ratio[np.logical_not(np.isnan(peak_ratio))] 


sort_ratio_index = np.argsort(peak_ratio)
print(sort_ratio_index)
ratio_threshold = 1.2  ## Retain data with ratio > 1.2
print(peak_ratio.shape)
print(peak_ratio[sort_ratio_index])
a = 0
for i in range(peak_ratio.shape[0]):
    if peak_ratio[sort_ratio_index[i]] > ratio_threshold:
        a = i
        break


qualified_ratio = sort_ratio_index[a:]
qualified_ratio = list(qualified_ratio)
qualified_ratio.sort()
df = df.iloc[qualified_ratio, :]
count = df.shape[0]
df.to_csv(data_dir + 'Cleaned_SERS_dataset.csv', index= False)


## Record Clean dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `cleaned`, 
                         `data_type` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}',
                       '{cleaned}', '{data_type}');""".format(
                file_name = 'Cleaned_SERS_dataset.csv', antibiotic=antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'Cleaned_SERS_dataset.csv', count = count,
                combine_id = RAW_id, cleaned = 1, data_type = 'dataset')
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()





## Statistic of Cleaned dataset

In [ ]:
## Record Clean Statistic into database

# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")

cursor.execute("SELECT LAST_INSERT_ID();")
cleaned_id = cursor.fetchall()[0][0]

sql_command = """SELECT `data_path` FROM `Preprocess_SERS`
                WHERE `id` = {cleaned_id};""".format(cleaned_id = cleaned_id)
cursor.execute(sql_command)
cleaned_path = cursor.fetchall()[0][0]


# READ Cleaned_dataset

df = pd.read_csv(cleaned_path, header = 0) #data_path = data_dir + 'Cleaned_SERS_dataset.csv'


# Counting
clean_count = df['label'].value_counts()
clean_count.to_csv(data_dir +'Cleaned_SERS_dataset_Count.csv', index= True)


sql_command = """INSERT INTO `Preprocess_SERS_Statistic`
                        (`file_name`,`preprocess_id`, `data_type`,`data_path`)
               VALUES ('{file_name}','{preprocess_id}', '{data_type}','{data_path}');""".format(
                file_name = 'Cleaned_SERS_dataset_Count.csv', 
                preprocess_id = cleaned_id,
                data_type = 'count',
                data_path = data_dir + 'Cleaned_SERS_dataset_Count.csv')
cursor.execute(sql_command)
mydb.commit()

## Mean of Cleaned SERS spectra

df.groupby('label').mean()
df.groupby('label').std()

df_mean= df.groupby('label').mean()
df_mean.to_csv(data_dir + 'Cleaned_SERS_dataset_mean.csv', index= True)

sql_command = """INSERT INTO `Preprocess_SERS_Statistic`
                        (`file_name`,`preprocess_id`, `data_type`,`data_path`)
               VALUES ('{file_name}','{preprocess_id}', '{data_type}','{data_path}');""".format(
                file_name = 'Cleaned_SERS_dataset_mean.csv', 
                preprocess_id = cleaned_id,
                data_type = 'mean',
                data_path = data_dir + 'Cleaned_SERS_dataset_mean.csv')
cursor.execute(sql_command)
mydb.commit()



## STD of Cleaned SERS spectra
df_std= df.groupby('label').std()
df_std.to_csv(data_dir + 'Cleaned_SERS_dataset_std.csv', index= True)

sql_command = """INSERT INTO `Preprocess_SERS_Statistic`
                        (`file_name`,`preprocess_id`, `data_type`,`data_path`)
               VALUES ('{file_name}','{preprocess_id}', '{data_type}','{data_path}');""".format(
                file_name = 'Cleaned_SERS_dataset_std.csv', 
                preprocess_id = cleaned_id,
                data_type = 'std',
                data_path = data_dir + 'Cleaned_SERS_dataset_std.csv')
cursor.execute(sql_command)
mydb.commit()

### For Datalabel representation 

In [ ]:
### For Datalabel representation 


# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")

df = pd.read_csv(cleaned_path, header = 0)

df['label'].replace({
'Ecoli-ATCC27662_Amp16_30min': '0', 
'Ecoli-BL21_Amp16_30min': '1', 
'Ecoli-BW25113_Amp16_30min':'2', 
'Ecoli-DH5alpha_Amp16_30min':'3',   
'Ecoli-DH5alpha-ampR_Amp16_30min':'4'}, inplace=True)

count = df.shape[0]
df.to_csv(data_dir + 'Labeled_SERS_dataset.csv', index= False)

## Record Lable dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `labeled` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{labeled}');""".format(
                file_name = 'Labeled_SERS_dataset.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'Labeled_SERS_dataset.csv', count = count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, labeled = 1)
cursor.execute(sql_command)
mydb.commit()



## Train test dataset split

In [ ]:
# Train test dataset split

## Directory
data_dir = 'Exp_Data/'

# experimental conditions
antibiotic = 'Amp'
anti_conc = 16
anti_time = '30min'


# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")


cursor.execute("""SELECT `data_path`,`id` FROM `Combine_SERS`
        WHERE (`antibiotic` = '{antibiotic}') AND (`anti_conc` = '{anti_conc}') AND (`anti_time` = '{anti_time}');""".format(
        antibiotic = antibiotic, anti_conc = anti_conc, anti_time = anti_time))
RAW_dataset = cursor.fetchall()
RAW_file = RAW_dataset[0][0]
RAW_id  = RAW_dataset[0][1]


sql_command = """SELECT `data_path` FROM `Preprocess_SERS`
                WHERE `file_name` = '{file_name}';""".format(
                    file_name = 'Labeled_SERS_dataset.csv')
cursor.execute(sql_command)
# cursor.fetchall()
labeled_path = cursor.fetchall()[0][0]



df = pd.read_csv(labeled_path, header = 0) #data_path = data_dir + 'Labeled_SERS_dataset.csv'
cols = list(df.columns)

# print(df['label'])

combine = df.to_numpy()
np.random.seed(400)
np.random.shuffle(combine)
print (combine.shape)

percent10 = int (combine.shape[0] * 0.1)
print(percent10)

test_set = combine[0:percent10,:]
train_set = combine[percent10:,:]

test_set = pd.DataFrame(test_set, columns=cols)
train_set = pd.DataFrame(train_set, columns=cols)

test_count = test_set.shape[0]
train_count = train_set.shape[0]

test_set.to_csv(data_dir  + 'SERS_testing.csv', index= False , header = True)
train_set.to_csv(data_dir + 'SERS_training.csv', index= False , header = True)

## Record Test dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `labeled`, `test` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{labeled}','{test}');""".format(
                file_name = 'SERS_testing.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'SERS_testing.csv', count = test_count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, labeled = 1, test = 1)
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()

## Record Train dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `labeled`, `train` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{labeled}','{train}');""".format(
                file_name = 'SERS_training.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'SERS_training.csv', count = train_count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, labeled = 1, train = 1)
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()




## Normalized 

In [ ]:
# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")


sql_command = """SELECT `data_path` FROM `Preprocess_SERS`
                WHERE `file_name` = '{file_name}';""".format(
                    file_name = 'Cleaned_SERS_dataset.csv')
cursor.execute(sql_command)
# cursor.fetchall()
cleaned_path = cursor.fetchall()[0][0]
df = pd.read_csv(cleaned_path, header = 0)  ## data_path = data_dir + 'Cleaned_SERS_dataset.csv'

#Normalization for each spectra
label = df.pop('label')
#df.pop('Unnamed: 0')
header_name = df.columns
df = df.to_numpy()
min_value = np.min(df, axis=1)
min_value = np.reshape(min_value, (-1, 1))
min_value = np.repeat(min_value, df.shape[1], axis=1)
max_value = np.max(df, axis=1)
max_value = np.reshape(max_value, (-1, 1))
max_value = np.repeat(max_value, df.shape[1], axis=1)
df = df - min_value
df = df / (max_value - min_value)
#Find Peak
df = pd.DataFrame(df, columns=header_name)
df = pd.concat([label,df], axis=1)
count = df.shape[0]
df.to_csv(data_dir + 'Cleaned_Nor_SERS_dataset.csv', index= False)

## Record Normalized dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `normalized` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{normalized}');""".format(
                file_name = 'Cleaned_Nor_SERS_dataset.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'Cleaned_Nor_SERS_dataset.csv', count = count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, normalized = 1)
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()



# df.groupby('label').mean()
# df.groupby('label').std()


cursor.execute("SELECT LAST_INSERT_ID();")
cleaned_nor_id = cursor.fetchall()[0][0]

## Record mean of Normalized dataset into database
df_mean= df.groupby('label').mean()
df_mean.to_csv(data_dir + 'Cleaned_Nor_SERS_dataset_mean.csv', index= True)

sql_command = """INSERT INTO `Preprocess_SERS_Statistic`
                        (`file_name`,`preprocess_id`, `data_type`,`data_path`)
               VALUES ('{file_name}','{preprocess_id}', '{data_type}','{data_path}');""".format(
                file_name = 'Cleaned_Nor_SERS_dataset_mean.csv', 
                preprocess_id = cleaned_nor_id,
                data_type = 'mean',
                data_path = data_dir + 'Cleaned_Nor_SERS_dataset_mean.csv')
cursor.execute(sql_command)
mydb.commit()


## Record std of Normalized dataset into database
df_std= df.groupby('label').std()
df_std.to_csv(data_dir + 'Cleaned_Nor_SERS_dataset_std.csv', index= True)

sql_command = """INSERT INTO `Preprocess_SERS_Statistic`
                        (`file_name`,`preprocess_id`, `data_type`,`data_path`)
               VALUES ('{file_name}','{preprocess_id}', '{data_type}','{data_path}');""".format(
                file_name = 'Cleaned_Nor_SERS_dataset_std.csv', 
                preprocess_id = cleaned_nor_id,
                data_type = 'std',
                data_path = data_dir + 'Cleaned_Nor_SERS_dataset_std.csv')
cursor.execute(sql_command)
mydb.commit()



### Normalized + label

In [ ]:
## Nor label

# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")

sql_command = """SELECT `data_path` FROM `Preprocess_SERS`
                WHERE `file_name` = '{file_name}';""".format(
                    file_name = 'Cleaned_Nor_SERS_dataset.csv')
cursor.execute(sql_command)
# cursor.fetchall()
cleaned_nor_path = cursor.fetchall()[0][0]


df = pd.read_csv(cleaned_nor_path, header = 0) ## data_path = data_dir + 'Cleaned_Nor_SERS_dataset.csv'

df['label'].replace({
'Ecoli-ATCC27662_Amp16_30min': '0', 
'Ecoli-BL21_Amp16_30min': '1', 
'Ecoli-BW25113_Amp16_30min':'2', 
'Ecoli-DH5alpha_Amp16_30min':'3',   
'Ecoli-DH5alpha-ampR_Amp16_30min':'4'}, inplace=True)

df.to_csv(data_dir + 'Labeled_Nor_SERS_dataset.csv', index= False)

## Record Lable dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `normalized`, `labeled` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{normalized}','{labeled}');""".format(
                file_name = 'Labeled_Nor_SERS_dataset.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'Labeled_Nor_SERS_dataset.csv', count = count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, normalized = 1,labeled = 1)
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()



## Normalized: Train test dataset split

In [ ]:
# Nor: Train test dataset split

# ensure database connection
cursor.close()
mydb.reconnect()
cursor = mydb.cursor(buffered=True)
cursor.execute("USE `SERS_ML_TEST`;")


## Directory
data_dir = 'Exp_Data/'

# experimental conditions
antibiotic = 'Amp'
anti_conc = 16
anti_time = '30min'


cursor.execute("""SELECT `data_path`,`id` FROM `Combine_SERS`
        WHERE (`antibiotic` = '{antibiotic}') AND (`anti_conc` = '{anti_conc}') AND (`anti_time` = '{anti_time}');""".format(
        antibiotic = antibiotic, anti_conc = anti_conc, anti_time = anti_time))
RAW_dataset = cursor.fetchall()
RAW_file = RAW_dataset[0][0]
RAW_id  = RAW_dataset[0][1]




sql_command = """SELECT `data_path` FROM `Preprocess_SERS`
                WHERE `file_name` = '{file_name}';""".format(
                    file_name = 'Labeled_Nor_SERS_dataset.csv')
cursor.execute(sql_command)
# cursor.fetchall()
labeled_nor_path = cursor.fetchall()[0][0]


df = pd.read_csv(labeled_nor_path, header = 0) ## data_path = data_dir + 'Labeled_Nor_SERS_dataset.csv'
cols = list(df.columns)

combine = df.to_numpy()
np.random.seed(400)
np.random.shuffle(combine)
print (combine.shape)

percent10 = int (combine.shape[0] * 0.1)
print(percent10)

test_set = combine[0:percent10,:]
train_set = combine[percent10:,:]

test_set = pd.DataFrame(test_set, columns=cols)
train_set = pd.DataFrame(train_set, columns=cols)

test_count = test_set.shape[0]
train_count = train_set.shape[0]

test_set.to_csv(data_dir  + 'SERS_Nor_testing.csv', index= False , header = True)
train_set.to_csv(data_dir + 'SERS_Nor_training.csv', index= False , header = True)


## Record Test Nor-dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `normalized`,`labeled`, `test` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{normalized}','{labeled}','{test}');""".format(
                file_name = 'SERS_Nor_testing.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'SERS_Nor_testing.csv', count = test_count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, normalized = 1, labeled = 1, test = 1)
print (sql_command)
cursor.execute(sql_command)
mydb.commit()




## Record Train Nor-dataset into Database
sql_command = """INSERT INTO `Preprocess_SERS`
                        (`file_name`,`antibiotic`, `anti_conc`,`anti_time`, 
                         `data_path`,`count`, `combine_id`, `data_type` ,
                         `cleaned`, `normalized`, `labeled`, `train` )
               VALUES ('{file_name}','{antibiotic}', '{anti_conc}','{anti_time}', 
                       '{data_path}','{count}', '{combine_id}','{data_type}',
                       '{cleaned}', '{normalized}', '{labeled}','{train}');""".format(
                file_name = 'SERS_Nor_training.csv', antibiotic = antibiotic, 
                anti_conc=anti_conc, anti_time = anti_time, 
                data_path = data_dir + 'SERS_Nor_training.csv', count = train_count,
                combine_id = RAW_id, data_type = 'dataset', 
                cleaned = 1, normalized = 1, labeled = 1, train = 1)
# print (sql_command)
cursor.execute(sql_command)
mydb.commit()


